# KPI Reporting Dashboard - Retail Sales Data Warehouse

This notebook generates key performance indicators (KPIs) and business metrics for executive reporting.

## Key Performance Indicators:
- Revenue metrics (Total, MoM, YoY growth)
- Customer metrics (Retention, Acquisition, LTV)
- Product metrics (Inventory turnover, Top sellers)
- Operational metrics (Order fulfillment, Returns)

## 1. Setup and Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# Import custom modules
import sys
sys.path.append('..')
from src.utils.db_connection import DatabaseConnection

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

print("Environment configured successfully")
print(f"Report Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Database Connection

In [ ]:
# Configure database connection
connection_params = {
    'db_type': 'postgresql',
    'host': 'localhost',
    'port': '5432',
    'database': 'retail_dw',
    'username': 'your_username',
    'password': 'your_password'
}

# db = DatabaseConnection(connection_params)

## 3. Revenue KPIs

In [ ]:
# Calculate revenue metrics
query_revenue = """
SELECT 
    DATE_TRUNC('month', sale_date) as month,
    SUM(total_amount) as total_revenue,
    COUNT(DISTINCT sale_id) as transaction_count,
    AVG(total_amount) as avg_transaction_value
FROM fact_sales
WHERE sale_date >= CURRENT_DATE - INTERVAL '12 months'
GROUP BY DATE_TRUNC('month', sale_date)
ORDER BY month
"""

# df_revenue = db.execute_query(query_revenue)
# print("Revenue KPIs calculated")
# df_revenue.head()

In [ ]:
# Visualize monthly revenue trend
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# # Revenue trend
# ax1.plot(df_revenue['month'], df_revenue['total_revenue'], marker='o', linewidth=2, markersize=8)
# ax1.set_title('Monthly Revenue Trend', fontsize=14, fontweight='bold')
# ax1.set_xlabel('Month')
# ax1.set_ylabel('Revenue ($)')
# ax1.grid(True, alpha=0.3)
# ax1.tick_params(axis='x', rotation=45)

# # Transaction count
# ax2.bar(df_revenue['month'], df_revenue['transaction_count'], color='steelblue', alpha=0.7)
# ax2.set_title('Monthly Transaction Volume', fontsize=14, fontweight='bold')
# ax2.set_xlabel('Month')
# ax2.set_ylabel('Number of Transactions')
# ax2.tick_params(axis='x', rotation=45)

# plt.tight_layout()
# plt.show()

## 4. Customer KPIs

In [ ]:
# Customer acquisition and retention metrics
query_customers = """
WITH customer_stats AS (
    SELECT 
        customer_key,
        COUNT(*) as purchase_count,
        SUM(total_amount) as lifetime_value,
        MIN(sale_date) as first_purchase,
        MAX(sale_date) as last_purchase
    FROM fact_sales
    GROUP BY customer_key
)
SELECT 
    COUNT(*) as total_customers,
    AVG(lifetime_value) as avg_customer_ltv,
    AVG(purchase_count) as avg_purchases_per_customer,
    COUNT(CASE WHEN last_purchase >= CURRENT_DATE - INTERVAL '90 days' THEN 1 END) as active_customers
FROM customer_stats
"""

# df_customers = db.execute_query(query_customers)
# print("Customer KPIs:")
# print(df_customers)

## 5. Product Performance KPIs

In [ ]:
# Top performing products
query_products = """
SELECT 
    p.product_name,
    p.category,
    COUNT(f.sale_id) as units_sold,
    SUM(f.total_amount) as total_revenue,
    AVG(f.unit_price) as avg_price
FROM fact_sales f
JOIN dim_product p ON f.product_key = p.product_key
WHERE f.sale_date >= CURRENT_DATE - INTERVAL '3 months'
GROUP BY p.product_name, p.category
ORDER BY total_revenue DESC
LIMIT 15
"""

# df_products = db.execute_query(query_products)
# print("Top 15 Products (Last 3 Months):")
# df_products.head()

In [ ]:
# Visualize product performance
# fig, ax = plt.subplots(figsize=(14, 8))
# bars = ax.barh(df_products['product_name'], df_products['total_revenue'], color='teal', alpha=0.7)
# ax.set_xlabel('Total Revenue ($)', fontsize=12)
# ax.set_ylabel('Product', fontsize=12)
# ax.set_title('Top 15 Products by Revenue (Last 3 Months)', fontsize=14, fontweight='bold')
# ax.grid(axis='x', alpha=0.3)
# plt.tight_layout()
# plt.show()

## 6. Growth Metrics

In [ ]:
# Calculate MoM and YoY growth
# df_revenue['revenue_mom_growth'] = df_revenue['total_revenue'].pct_change() * 100
# df_revenue['revenue_yoy_growth'] = df_revenue['total_revenue'].pct_change(periods=12) * 100

# print("Growth Metrics:")
# print(df_revenue[['month', 'total_revenue', 'revenue_mom_growth', 'revenue_yoy_growth']].tail())

## 7. Sales Channel Performance

In [ ]:
# Sales by channel (if available)
query_channels = """
SELECT 
    s.store_name,
    s.store_type,
    COUNT(f.sale_id) as transaction_count,
    SUM(f.total_amount) as total_revenue
FROM fact_sales f
JOIN dim_store s ON f.store_key = s.store_key
WHERE f.sale_date >= CURRENT_DATE - INTERVAL '1 month'
GROUP BY s.store_name, s.store_type
ORDER BY total_revenue DESC
"""

# df_channels = db.execute_query(query_channels)
# print("Sales Channel Performance:")
# df_channels.head()

## 8. Executive Summary Dashboard

In [ ]:
# Create comprehensive KPI dashboard
def create_kpi_card(value, label, change=None):
    """Display KPI in a formatted card"""
    change_str = f" ({change:+.1f}%)" if change is not None else ""
    print(f"\n{'='*50}")
    print(f"{label.upper()}")
    print(f"{value:,.2f}{change_str}")
    print(f"{'='*50}")

# Example KPI cards
# create_kpi_card(1250000, "Total Revenue (MTD)", 12.5)
# create_kpi_card(4250, "Active Customers", 8.3)
# create_kpi_card(294.50, "Average Order Value", -2.1)
# create_kpi_card(89.5, "Customer Satisfaction", 1.2)

## 9. Automated Reporting

In [ ]:
# Export KPI report to CSV
# report_date = datetime.now().strftime('%Y%m%d')
# output_file = f'../data/processed/kpi_report_{report_date}.csv'

# Create comprehensive report
# kpi_report = pd.DataFrame({
#     'Metric': ['Total Revenue', 'Total Transactions', 'Active Customers', 'Avg Order Value'],
#     'Value': [1250000, 5420, 4250, 294.50],
#     'MoM_Change': [12.5, 8.3, 5.2, -2.1]
# })

# kpi_report.to_csv(output_file, index=False)
# print(f"KPI report exported to: {output_file}")

## 10. Alerts and Recommendations

In [ ]:
# Define alert thresholds and generate recommendations
def generate_alerts(kpis):
    """
    Generate alerts based on KPI thresholds
    """
    alerts = []
    
    # Example alert logic
    # if kpis['revenue_growth'] < 0:
    #     alerts.append("⚠️ Revenue decline detected")
    # if kpis['customer_retention'] < 70:
    #     alerts.append("⚠️ Customer retention below target")
    # if kpis['inventory_turnover'] < 4:
    #     alerts.append("⚠️ Slow inventory movement")
    
    return alerts

# alerts = generate_alerts(kpis)
# for alert in alerts:
#     print(alert)

## Conclusion

This KPI dashboard provides real-time insights into business performance. Schedule this notebook to run automatically for regular reporting.